In [19]:
#!pip install scikit-surprise
from surprise import Dataset, Reader, SVD
import pandas as pd

# Carga de datos usuario, producto y su calificación
ratings_data = {
    'user_id': [1, 1, 2, 2, 3, 3, 3, 4, 4, 4],
    'item_id': [1, 2, 3, 4, 1, 2, 3, 1, 2, 3],
    'rating': [5, 3, 5, 2, 4, 3, 5, 2, 4, 2]
}

df = pd.DataFrame(ratings_data)

# Rango de las calificaciones
reader = Reader(rating_scale=(1, 5))

# Creando un df con los datos
data = Dataset.load_from_df(df[['user_id', 'item_id', 'rating']], reader)

# Entrenamiento SVD
trainset = data.build_full_trainset()
model = SVD()
model.fit(trainset)

# Función para obtener las mejores recomendaciones para un usuario dado
def get_top_recommendations(user_id, n=2):
    items = []
    # Obtener todos los ítems que el usuario no ha calificado
    items_not_rated_by_user = [item for item in range(1, trainset.n_items + 1)
                               if item not in trainset.ur[user_id]]
    # Obtener la predicción de calificación para cada ítem no calificado
    for item_id in items_not_rated_by_user:
        predicted_rating = model.predict(user_id, item_id).est
        items.append((item_id, predicted_rating))
    # Ordenar las predicciones y devolver las mejores n recomendaciones
    top_items = sorted(items, key=lambda x: x[1], reverse=True)[:n]
    return [item[0] for item in top_items]

###############################################################################
# Ejemplo de uso, recomendación para el usuario dado
###############################################################################
user_id = 2
top_recommendations = get_top_recommendations(user_id)
print("Top 2 recomendaciones para el usuario", user_id, ":", top_recommendations)


Top 2 recomendaciones para el usuario 2 : [3, 1]
